# Project Progress

## Initial Imports

In [17]:
import sys
from pathlib import Path

root_dir = Path().absolute().parent.parent
sys.path.append(str(root_dir))

In [18]:
# Warning control
import json
import os
import warnings
import yaml
from dotenv import load_dotenv

from crewai import Agent, Task, Crew

warnings.filterwarnings('ignore')

load_dotenv()

True

## Load OpenAI Model

In [19]:
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'

## Loading Tasks and Agents YAML files

In [20]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

## Creating Custom Tools

In [42]:
from typing import Type

import asana
from crewai_tools import BaseTool
from pydantic import BaseModel, Field


class ProjectTasksFetcherTool(BaseTool):
    name: str = "Asana Project Tasks Fetcher"
    description: str = "Fetches all tasks and their details from an Asana project"
    
    def _run(self) -> dict:
        """
        Fetch all tasks from the specified Asana project.
        """
        configuration = asana.Configuration()
        configuration.access_token = os.environ['ASANA_ACCESS_TOKEN']
        api_client = asana.ApiClient(configuration)
        
        tasks_api = asana.TasksApi(api_client)
        project_gid = os.environ['ASANA_PROJECT_GID']
        
        opts = {
            'completed_since': 'now',
            'limit': 100,
            'opt_fields': (
                "gid,name,notes,completed,completed_at,due_on,due_at,"
                "assignee.name,tags.name,memberships.section.name"
            )
        }
        
        try:
            tasks = tasks_api.get_tasks_for_project(project_gid, opts)
            return [task for task in tasks]
        except asana.rest.ApiException as e:
            return {"error": f"Failed to fetch project tasks: {str(e)}"}

class TaskInput(BaseModel):
        """Input schema for TaskDataFetcherTool."""
        task_gid: str = Field(..., description="Asana task GID to fetch details for")

class TaskDataFetcherTool(BaseTool):
    name: str = "Asana Task Data Fetcher"
    description: str = "Fetches detailed data for a specific Asana task. Requires task_gid as input."
    args_schema: Type[BaseModel] = TaskInput  # We still need this for the task_gid input

    def _run(self, task_gid: str) -> dict:
        """
        Fetch detailed data for a specific task.
        """
        configuration = asana.Configuration()
        configuration.access_token = os.environ['ASANA_ACCESS_TOKEN']
        api_client = asana.ApiClient(configuration)
        
        tasks_api = asana.TasksApi(api_client)
        
        opts = {
            'opt_fields': (
                "name,notes,completed,completed_at,due_on,due_at,"
                "assignee.name,tags.name,memberships.section.name,"
                "html_notes,followers.name,num_subtasks"
            )
        }
        
        try:
            task = tasks_api.get_task(task_gid, opts)
            return task
        except asana.rest.ApiException as e:
            return {"error": f"Failed to fetch task data: {str(e)}"}

## Create Crew, Agents and Tasks

In [43]:
access_token = os.environ['ASANA_ACCESS_TOKEN']
project_gid = os.environ['ASANA_PROJECT_GID']

# Creating Agents
data_collection_agent = Agent(
  config=agents_config['data_collection_agent'],
  tools=[
    ProjectTasksFetcherTool(),
    TaskDataFetcherTool()
  ]
)

analysis_agent = Agent(
  config=agents_config['analysis_agent']
)

# Creating Tasks
data_collection = Task(
  config=tasks_config['data_collection'],
  agent=data_collection_agent
)

data_analysis = Task(
  config=tasks_config['data_analysis'],
  agent=analysis_agent
)

report_generation = Task(
  config=tasks_config['report_generation'],
  agent=analysis_agent,
)

# Creating Crew
crew = Crew(
  agents=[
    data_collection_agent,
    analysis_agent
  ],
  tasks=[
    data_collection,
    data_analysis,
    report_generation
  ],
  verbose=True
)

2024-12-16 17:46:09,905 - 11692 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


## Kickoff Crew

In [44]:
# Kick off the crew and execute the process
result = crew.kickoff()

# Agent: Data Collection Specialist
## Task: Create an initial understanding of the project, its main features and the team working on it. Use the Project Tasks Fetcher Tool tool to gather basis tasks' data for an Asana project and the Task Data Fetcher Tool to gather task details.



# Agent: Data Collection Specialist
## Thought: I need to start by gathering all the tasks and their details from the Asana project to create a complete report on the project, its main features, and the team working on it.
## Using tool: Asana Project Tasks Fetcher
## Tool Input: 
"{}"
## Tool Output: 
[{'gid': '1207419414803639', 'assignee': None, 'completed': False, 'completed_at': None, 'due_at': None, 'due_on': None, 'memberships': [{'section': {'gid': '1205501845882202', 'name': 'Review Quarterly'}}], 'name': 'NA - check video Authority Hacker for new business model to follow', 'notes': 'https://www.youtube.com/watch?v=K_Y9MYiW0Go\n\nhttps://app.asana.com/app/asana/-/get_asset?asset_id=12074194148036

## Usage Metrics and Costs

Let’s see how much it would cost each time if this crew runs at scale.

In [47]:
import pandas as pd

costs = 0.150 * (crew.usage_metrics.prompt_tokens + crew.usage_metrics.completion_tokens) / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([crew.usage_metrics.dict()])
df_usage_metrics

Total costs: $0.0042


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,28251,26070,18176,2181,10


## Report

In [48]:
from IPython.display import Markdown

markdown  = result.raw
Markdown(markdown)

# Sprint Report: Project Progress Analysis

## Sprint Overview
The project focuses on developing a business model centered on content creation related to kink and sex work, utilizing tools like Webflow for efficient blog management and lead generation. Current analysis indicates significant blockers and delays impeding progress. 

## Task Summary
| Task                                          | Status     | Blocking Issue                                                                                       |
|-----------------------------------------------|------------|-----------------------------------------------------------------------------------------------------|
| Authority Hacker Video Analysis                | Incomplete | Foundational knowledge is lacking, hampering effective business model development.                  |
| Database Upgrade                               | Incomplete | Essential upgrade for efficient data processing; delays impact website management and offerings.    |
| CLTV Adjustment                               | Incomplete | Accurate financial projections require this adjustment; delays strategic planning.                  |
| Blog and Lead Magnet Development               | Incomplete | Delay in content creation affects lead generation, crucial for business model success.              |
| Keyword Research and Volume Analysis           | Incomplete | Incomplete research inhibits content strategy, risking lower search visibility and engagement.      |
| Personal Website Development                   | Incomplete | Absence of personal website limits branding and promotional activities.                             |

## Identified Issues and Blockers
The project is currently experiencing multiple issues that require immediate attention to prevent further delays:
- **Lack of Knowledge**: The team needs insights from the Authority Hacker video to make informed decisions.
- **Database Dependency**: Without an upgraded database, operations are hindered, affecting overall project functionality.
- **Financial Accuracy**: Delay in adjusting CLTV is affecting financial strategy, risking future projections.
- **Content Gaps**: Incomplete blogs and lead magnets hinder lead generation processes critical for project sustainability.
- **Keyword Strategy**: Incomplete keyword research jeopardizes the content strategy, leading to potential visibility issues.
- **Branding Void**: The absence of a personal website restricts branding opportunities for team members.

## Progress and Delays
The sprint has seen considerable delays due to the above blockers, with no completed tasks to report. Team engagement appears lacking despite Jonatan Snyders' oversight. Overall, progress is stagnant, and the original project timeline is at risk.

## Team Performance Overview
While Jonatan Snyders has been proactive in overseeing the project, the lack of timely task completion from the rest of the team has resulted in significant stagnation. This underperformance has hindered project momentum and requires immediate remediation.

## Action Items and Recommendations
1. **Prioritize Completion of Blocked Tasks**: 
   - Schedule and enforce a timeline for completing essential tasks such as the Authority Hacker video analysis and database upgrade.
2. **Establish Clear Deadlines**: 
   - Set firm deadlines for each incomplete task to encourage accountability among team members.
3. **Enhance Collaboration**: 
   - Foster collaborative efforts between team members to ensure tasks are driven to completion.
4. **Regular Status Updates**: 
   - Implement short, regular check-ins to monitor progress and address emerging issues swiftly.
5. **Skill Development**: 
   - Identify knowledge gaps within the team and provide necessary training or resources to enhance skills and efficiency.

## Conclusion
The current state of the project requires immediate attention to the identified blockers and delays. A structured approach with clear deadlines and enhanced collaboration will be crucial in restoring momentum and ensuring project objectives are achieved. Addressing these bottlenecks is essential for significant progress and the project's overall success. Immediate action and a commitment to follow-through will pave the way for improved outcomes moving forward.